# Analysis of Multi-layer resnet50 from MOSAIKS

## Prepare GeoLife data loader

In [1]:
import os
import sys
import time
import inspect
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
import timm

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.image import extract_patches_2d

CURR_DIR = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
PARENT_DIR = os.path.dirname(CURR_DIR)
sys.path.insert(0, "/home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/remote_sensing")

from dataset.pytorch_dataset import GeoLifeCLEF2022Dataset
from torch.utils.data import random_split, DataLoader

random_state = np.random.RandomState(0)
num_species= 17037

/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/ffcv2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_kmeans    = timm.create_model(model_name = 'resnet50', pretrained = True)
model_kmeans.fc = nn.Linear(2048, num_species)

In [3]:
ckpt_path = "/home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/ckpts/resnet50_mosaiks_kmeans.ckpt"

In [4]:
model_kmeans.load_state_dict(torch.load(ckpt_path))

<All keys matched successfully>

In [5]:
# for name, param in model_kmeans.named_parameters():
#     print(name, '\t\t\t', param.data.mean(), param.data.std())

In [6]:
conv_idxs = []
idx = 0
for name, param in model_kmeans.named_parameters():
    if 'conv' in name:
        conv_idxs.append(idx)
#         print(name, '---------------\t', param.shape)
#         print(idx)
#         conv_idxs[f"conv_layer_{idx}"] = param.shape
    
    idx += 1

In [7]:
filter_scale = 1e-3
for name, param in model_kmeans.named_parameters():
    x = param.detach().numpy()
    norm_param = (x - x.mean())/(x.std())
    param.data = torch.from_numpy(norm_param)
    print(name, '\t', param.detach().mean(), param.detach().std())

conv1.weight 	 tensor(5.2712e-09) tensor(1.0001)
bn1.weight 	 tensor(1.3411e-07) tensor(1.0079)
bn1.bias 	 tensor(-1.0245e-08) tensor(1.0079)
layer1.0.conv1.weight 	 tensor(-8.8476e-09) tensor(1.0001)
layer1.0.bn1.weight 	 tensor(3.3528e-08) tensor(1.0079)
layer1.0.bn1.bias 	 tensor(9.3132e-09) tensor(1.0079)
layer1.0.conv2.weight 	 tensor(-3.3114e-09) tensor(1.0000)
layer1.0.bn2.weight 	 tensor(1.3318e-07) tensor(1.0079)
layer1.0.bn2.bias 	 tensor(2.0489e-08) tensor(1.0079)
layer1.0.conv3.weight 	 tensor(-2.3283e-09) tensor(1.0000)
layer1.0.bn3.weight 	 tensor(-1.8626e-09) tensor(1.0020)
layer1.0.bn3.bias 	 tensor(-5.5879e-09) tensor(1.0020)
layer1.0.downsample.0.weight 	 tensor(1.1874e-08) tensor(1.0000)
layer1.0.downsample.1.weight 	 tensor(-6.3330e-08) tensor(1.0020)
layer1.0.downsample.1.bias 	 tensor(-5.5879e-09) tensor(1.0020)
layer1.1.conv1.weight 	 tensor(6.9849e-10) tensor(1.0000)
layer1.1.bn1.weight 	 tensor(-1.4901e-07) tensor(1.0079)
layer1.1.bn1.bias 	 tensor(-1.3970e-08)

In [18]:
PATH = "/home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/ckpts/resnet50_mosaiks_kmeans_normalized.ckpt"
torch.save(model_kmeans.state_dict(), PATH)

In [11]:
list(model.named_parameters())[0][1].data.min(), list(model.named_parameters())[0][1].data.max()

(tensor(-1.9258), tensor(2.7969))

## extract all conv layers

In [12]:
conv_idxs = []
idx = 0
for name, param in model.named_parameters():
    if 'conv' in name:
        conv_idxs.append(idx)
#         print(name, '---------------\t', param.shape)
#         print(idx)
#         conv_idxs[f"conv_layer_{idx}"] = param.shape
    
    idx += 1

In [13]:
# del conv_idxs['conv_layer_0']
conv_idxs

[0,
 3,
 6,
 9,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 45,
 48,
 51,
 54,
 57,
 60,
 63,
 66,
 69,
 72,
 75,
 78,
 84,
 87,
 90,
 93,
 96,
 99,
 102,
 105,
 108,
 111,
 114,
 117,
 120,
 123,
 126,
 129,
 132,
 135,
 141,
 144,
 147,
 150,
 153,
 156]

## with hooks

In [14]:
features_dim = {}
def get_features(name):
    def hook(model, input, output):
        features_dim[name] = output.detach()
    return hook

In [15]:
model.act1.register_forward_hook(get_features('conv_layer_0'))  #model.layer1[1].act2

model.layer1[0].act1.register_forward_hook(get_features('conv_layer_3'))
model.layer1[0].act2.register_forward_hook(get_features('conv_layer_6'))
model.layer1[0].act3.register_forward_hook(get_features('conv_layer_9'))

model.layer1[1].act1.register_forward_hook(get_features('conv_layer_15'))
model.layer1[1].act2.register_forward_hook(get_features('conv_layer_18'))
model.layer1[1].act3.register_forward_hook(get_features('conv_layer_21'))

model.layer1[2].act1.register_forward_hook(get_features('conv_layer_24'))
model.layer1[2].act2.register_forward_hook(get_features('conv_layer_27'))
model.layer1[2].act3.register_forward_hook(get_features('conv_layer_30'))

model.layer2[0].act1.register_forward_hook(get_features('conv_layer_33'))
model.layer2[0].act2.register_forward_hook(get_features('conv_layer_36'))
model.layer2[0].act3.register_forward_hook(get_features('conv_layer_39'))

model.layer2[1].act1.register_forward_hook(get_features('conv_layer_45'))
model.layer2[1].act2.register_forward_hook(get_features('conv_layer_48'))
model.layer2[1].act3.register_forward_hook(get_features('conv_layer_51'))

model.layer2[2].act1.register_forward_hook(get_features('conv_layer_54'))
model.layer2[2].act2.register_forward_hook(get_features('conv_layer_57'))
model.layer2[2].act3.register_forward_hook(get_features('conv_layer_60'))

model.layer2[3].act1.register_forward_hook(get_features('conv_layer_63'))
model.layer2[3].act2.register_forward_hook(get_features('conv_layer_66'))
model.layer2[3].act3.register_forward_hook(get_features('conv_layer_69'))

model.layer3[0].act1.register_forward_hook(get_features('conv_layer_72'))
model.layer3[0].act2.register_forward_hook(get_features('conv_layer_75'))
model.layer3[0].act3.register_forward_hook(get_features('conv_layer_78'))

model.layer3[1].act1.register_forward_hook(get_features('conv_layer_84'))
model.layer3[1].act2.register_forward_hook(get_features('conv_layer_87'))
model.layer3[1].act3.register_forward_hook(get_features('conv_layer_90'))
    
model.layer3[2].act1.register_forward_hook(get_features('conv_layer_93'))
model.layer3[2].act2.register_forward_hook(get_features('conv_layer_96'))
model.layer3[2].act3.register_forward_hook(get_features('conv_layer_99'))
    
model.layer3[3].act1.register_forward_hook(get_features('conv_layer_102'))
model.layer3[3].act2.register_forward_hook(get_features('conv_layer_105'))
model.layer3[3].act3.register_forward_hook(get_features('conv_layer_108'))

model.layer3[4].act1.register_forward_hook(get_features('conv_layer_111'))
model.layer3[4].act2.register_forward_hook(get_features('conv_layer_114'))
model.layer3[4].act3.register_forward_hook(get_features('conv_layer_117'))

model.layer3[5].act1.register_forward_hook(get_features('conv_layer_120'))
model.layer3[5].act2.register_forward_hook(get_features('conv_layer_123'))
model.layer3[5].act3.register_forward_hook(get_features('conv_layer_126'))    

model.layer4[0].act1.register_forward_hook(get_features('conv_layer_129'))
model.layer4[0].act2.register_forward_hook(get_features('conv_layer_132'))
model.layer4[0].act3.register_forward_hook(get_features('conv_layer_135'))    

model.layer4[1].act1.register_forward_hook(get_features('conv_layer_141'))
model.layer4[1].act2.register_forward_hook(get_features('conv_layer_144'))
model.layer4[1].act3.register_forward_hook(get_features('conv_layer_147'))    

model.layer4[2].act1.register_forward_hook(get_features('conv_layer_150'))
model.layer4[2].act2.register_forward_hook(get_features('conv_layer_153'))
model.layer4[2].act3.register_forward_hook(get_features('conv_layer_156'))    

In [16]:
for layer_idx, conv_idx in enumerate(conv_idxs):

    if layer_idx+1 == len(conv_idxs):
        break
        
    print(f'kmeans for output of conv layer#{conv_idx}')
    
    curr_param_idx = conv_idxs[layer_idx+1]
    curr_param_sz = list(model.named_parameters())[curr_param_idx][1].data.shape
    
    curr_feat_dim = list(model.named_parameters())[conv_idx][1].shape
    num_feats = curr_param_sz[0]
    num_ch = curr_param_sz[1]
    patch_size = (curr_param_sz[2], curr_param_sz[3])
    num_iters = 3   # The online learning part: cycle over the whole dataset 6 times
    max_patches = int(num_feats/10)

    print(f'current feature map dim: {curr_feat_dim}')
    print(f'Initializing parameter#{curr_param_idx} with size: {curr_param_sz}')

    print(f'num_feats:{num_feats}, num_ch:{num_ch}, patch_size:{patch_size}')
    kmeans = MiniBatchKMeans(n_clusters=num_feats, 
                             random_state=random_state,
    #                          max_no_improvement=5,
    #                          tol=0.01,
    #                          max_iter=5,
    #                          batch_size = kmeans_bs,
                             verbose=True)

    geo_train_loader = DataLoader(
                    geo_train_dataset,
                    batch_size=batch_size,
                    num_workers=num_workers,
                    shuffle=True,
                    pin_memory=True,
                )

    index = 0
    buffer = []

    for _ in range(num_iters):
        for batch in geo_train_loader:
            patches, target, meta = batch

            _ = model(patches['rgb'])
            curr_feats = features_dim[f'conv_layer_{conv_idx}'].numpy() #.cpu().numpy())
            curr_feats = curr_feats.squeeze(0)
            curr_feats = np.einsum('ijk->jki', curr_feats)

            data = extract_patches_2d(curr_feats, patch_size, max_patches=max_patches, random_state=random_state)


            data = np.reshape(data, (len(data), -1))
            buffer.append(data)
            index += 1
            if index % int(len(geo_train_loader)/10) == 0:
        #         print(data)
                data = np.concatenate(buffer, axis=0)
                data -= np.mean(data, axis=0)
                data /= np.std(data, axis=0)
                if np.any(np.isnan(data)):
                    data = np.nan_to_num(data)
                kmeans.partial_fit(data)
                buffer = []
            if index % 10000 == 0:
                print("Partial fit of %4i out of %i" % (index, num_iters * len(geo_train_loader)))


    
    # change the weights of the corresponding conv layer
    list(model.named_parameters())[curr_param_idx][1].data = torch.from_numpy(
        kmeans.cluster_centers_.reshape(
                                            num_feats, 
                                            num_ch, 
                                            patch_size[0], 
                                            patch_size[1])
                                        )

    # save the model
    PATH = "/home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/ckpts/resnet50_mosaiks_kmeans.pt"
    torch.save(model.state_dict(), PATH)


kmeans for output of conv layer#0
current feature map dim: torch.Size([64, 3, 7, 7])
Initializing parameter#3 with size: torch.Size([64, 64, 1, 1])
num_feats:64, num_ch:64, patch_size:(1, 1)
kmeans for output of conv layer#3
current feature map dim: torch.Size([64, 64, 1, 1])
Initializing parameter#6 with size: torch.Size([64, 64, 3, 3])
num_feats:64, num_ch:64, patch_size:(3, 3)


/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebra

kmeans for output of conv layer#6
current feature map dim: torch.Size([64, 64, 3, 3])
Initializing parameter#9 with size: torch.Size([256, 64, 1, 1])
num_feats:256, num_ch:64, patch_size:(1, 1)
kmeans for output of conv layer#9
current feature map dim: torch.Size([256, 64, 1, 1])
Initializing parameter#15 with size: torch.Size([64, 256, 1, 1])
num_feats:64, num_ch:256, patch_size:(1, 1)
[MiniBatchKMeans] Reassigning 3 cluster centers.


/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide


kmeans for output of conv layer#15
current feature map dim: torch.Size([64, 256, 1, 1])
Initializing parameter#18 with size: torch.Size([64, 64, 3, 3])
num_feats:64, num_ch:64, patch_size:(3, 3)
[MiniBatchKMeans] Reassigning 5 cluster centers.


...

kmeans for output of conv layer#150
current feature map dim: torch.Size([512, 2048, 1, 1])
Initializing parameter#153 with size: torch.Size([512, 512, 3, 3])
num_feats:512, num_ch:512, patch_size:(3, 3)


/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebra

[MiniBatchKMeans] Reassigning 288 cluster centers.


/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in true_divide
/home/mila/s/sara.ebra

kmeans for output of conv layer#153
current feature map dim: torch.Size([512, 512, 3, 3])
Initializing parameter#156 with size: torch.Size([2048, 512, 1, 1])
num_feats:2048, num_ch:512, patch_size:(1, 1)
